**Counts the lines in a file.**

In [9]:
!wc -l goodreads_books.json.gz

 7588375 goodreads_books.json.gz


**Show me the size of all the files in the directory and then search only for the specific file (goodreads books).**

In [10]:
!ls -lh goodreads_books.json.gz

-rw-r--r--@ 1 britneyagius  staff   1.9G Mar 16 13:24 goodreads_books.json.gz


**Stream the file without unzipping**